# Benchmark Evaluation: Standard Datasets

**Goal:** Evaluate single-model and multi-agent strategies on established benchmarks

**Date:** [Fill in]  
**Author:** Leif Haven Martinson  

## Why Benchmarks?

- **Reproducibility**: Compare to published baselines
- **Validity**: Prove multi-agent helps on well-studied tasks
- **Generalization**: Test across diverse problem types
- **Calibration**: Understand if improvements are meaningful

## Available Benchmarks

1. **GSM8K** - Grade school math word problems (multi-step reasoning)
2. **MMLU** - Multitask language understanding (57 subjects)
3. **TruthfulQA** - Truthfulness and avoiding misconceptions
4. **ARC** - Science reasoning questions

## Research Questions

- Does debate improve accuracy on math reasoning (GSM8K)?
- Does consensus reduce hallucinations (TruthfulQA)?
- How do local models compare to API baselines?
- What's the cost/performance tradeoff?

In [ ]:
# Setup
import sys
sys.path.append('../code')

from harness import (
    load_benchmark,
    get_baseline_scores,
    BENCHMARKS,
    run_strategy,
    ExperimentConfig,
    ExperimentResult,
    get_tracker
)
from harness.defaults import DEFAULT_MODEL, DEFAULT_PROVIDER

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

%matplotlib inline
sns.set_style('whitegrid')

print("✅ Setup complete")
print(f"\nAvailable benchmarks: {list(BENCHMARKS.keys())}")

## Configuration

**CUSTOMIZE HERE:** Choose benchmark and strategies to test

# ========================================
# BENCHMARK CONFIGURATION
# ========================================

# Which benchmark to run
BENCHMARK_NAME = "gsm8k"  # 🔧 CHANGE: "gsm8k", "mmlu", "truthfulqa", "arc"

# How many tasks to evaluate (None = all, start with 10-20 for testing)
NUM_TASKS = 5  # 🔧 CHANGE: Start small, increase for full eval

# Random seed for reproducibility
SEED = 42

# Model configuration
PROVIDER = DEFAULT_PROVIDER
MODEL = DEFAULT_MODEL

# ========================================
# STRATEGIES TO COMPARE
# ========================================

# Define which strategies to test
# Each entry: (strategy_name, kwargs)
STRATEGIES_TO_TEST = [
    # Baseline
    ("single", {
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
    }),

    # Debate with 2 agents, 1 round
    ("debate", {
        "n_debaters": 2,
        "n_rounds": 1,
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
    }),

    # Debate with 3 agents, 2 rounds
    ("debate", {
        "n_debaters": 3,
        "n_rounds": 2,
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
    }),

    # Consensus (no judge)
    ("consensus", {
        "n_agents": 3,
        "n_rounds": 2,
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
    }),

    # Self-consistency
    ("self_consistency", {
        "n_samples": 5,
        "provider": PROVIDER,
        "model": MODEL,
        "temperature": 0.8
    }),

    # ========================================
    # CUSTOM MENTAL MODEL STRATEGIES (ACTIVE)
    # ========================================

    # Design Critique - Uses custom CRITIQUE_PANEL from cell above
    # 🔧 Edit the CRITIQUE_PANEL cell above to customize critics
    ("design_critique", {
        "n_iterations": 2,
        "critique_panel": CRITIQUE_PANEL,  # Uses your custom panel!
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": True  # 🔧 Set to True to see streaming output
    }),

    # Interdisciplinary Team - Uses custom EXPERT_TEAM from cell above
    # 🔧 Edit the EXPERT_TEAM cell above to customize team members
    ("interdisciplinary_team", {
        "refinement_rounds": 1,
        "expert_team": EXPERT_TEAM,  # Uses your custom team!
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": True  # 🔧 Set to True to see streaming output
    }),
]

print(f"✅ Configuration:")
print(f"   Benchmark: {BENCHMARK_NAME}")
print(f"   Tasks: {NUM_TASKS if NUM_TASKS else 'all'}")
print(f"   Strategies to test: {len(STRATEGIES_TO_TEST)}")
print(f"   Model: {MODEL} ({PROVIDER})")

In [ ]:
# ========================================
# DESIGN CRITIQUE PANEL - EDIT THESE PERSONAS
# ========================================

CRITIQUE_PANEL = [
    {
        "name": "Systems Designer",
        "focus": "System architecture and holistic design",
        "criteria": """You are a Systems Designer who thinks about the big picture and interconnections.
Evaluate:
- Does the solution consider the whole system and its parts?
- Are relationships between components clear?
- Is there coherence between different elements?
- Does the design scale and adapt to different contexts?
Focus on holistic thinking, interconnections, and systemic coherence."""
    },
    {
        "name": "Visual Craft Specialist",
        "focus": "Visual clarity, aesthetics, and presentation",
        "criteria": """You are a Visual Craft Specialist focused on how information is presented and perceived.
Evaluate:
- Is information presented clearly and visually comprehensible?
- Is there good hierarchy and structure in the presentation?
- Are concepts illustrated or explained in ways that are easy to visualize?
- Does the format enhance understanding?
Focus on clarity of presentation, visual thinking, and aesthetic quality."""
    },
    {
        "name": "AI Specialist",
        "focus": "AI/ML capabilities, limitations, and best practices",
        "criteria": """You are an AI Specialist with deep knowledge of AI systems, capabilities, and limitations.
Evaluate:
- Are claims about AI accurate and grounded in current capabilities?
- Are limitations and potential issues with AI acknowledged?
- Are AI-related recommendations practical and informed?
- Is the approach aligned with AI best practices?
Focus on technical accuracy regarding AI/ML, practical feasibility, and responsible AI considerations."""
    },
    {
        "name": "Human-Computer Interaction Expert",
        "focus": "User experience, usability, and human factors",
        "criteria": """You are an HCI Expert focused on how humans interact with systems and information.
Evaluate:
- Is the solution usable and accessible to the target audience?
- Are user needs and cognitive limitations considered?
- Is interaction intuitive and aligned with mental models?
- Are there potential usability issues or barriers?
Focus on user-centered design, accessibility, cognitive ergonomics, and interaction patterns."""
    },
    {
        "name": "IDEO Design Thinking Facilitator",
        "focus": "Human-centered innovation and creative problem-solving",
        "criteria": """You are an IDEO-trained Design Thinking practitioner emphasizing empathy, ideation, and iteration.
Evaluate:
- Does the solution demonstrate empathy for user needs and pain points?
- Is there creative thinking and exploration of possibilities?
- Are assumptions tested or validated?
- Is the approach iterative and open to refinement?
Focus on empathy, creative exploration, prototyping mindset, and bias toward action."""
    },
]

# ========================================
# INTERDISCIPLINARY TEAM - EDIT THESE PERSONAS
# ========================================

EXPERT_TEAM = [
    {
        "name": "Product Manager",
        "role": "Product Management",
        "perspective": "User needs, business value, roadmap, and strategic priorities",
        "system_prompt": """You are a Product Manager responsible for defining what to build and why.

Your mission: Ensure the solution creates real user value while achieving business objectives.

Focus on:
- User needs, pain points, and jobs-to-be-done
- Business impact, ROI, and strategic alignment
- Feature prioritization and tradeoffs
- Market fit and competitive positioning
- Success metrics and measurable outcomes
- Feasibility vs. value vs. risk assessment

Analyze problems by asking:
- What user problem does this solve?
- What is the business value?
- How do we measure success?
- What are the must-haves vs. nice-to-haves?
- What are the risks and mitigations?

Bring a balanced perspective that bridges user needs, business goals, and technical reality."""
    },
    {
        "name": "Software Engineer",
        "role": "Engineering",
        "perspective": "Technical implementation, architecture, scalability, and feasibility",
        "system_prompt": """You are a Software Engineer responsible for building and shipping reliable systems.

Your mission: Deliver technically sound solutions that are maintainable, scalable, and feasible within constraints.

Focus on:
- Technical feasibility and implementation complexity
- System architecture and design patterns
- Scalability, performance, and reliability
- Security, data integrity, and edge cases
- Technical debt and long-term maintainability
- Development velocity and engineering resources
- Integration with existing systems

Analyze problems by asking:
- Is this technically feasible?
- What is the implementation complexity?
- What are the technical risks?
- How does this scale?
- What are the dependencies and blockers?
- What technical debt are we taking on?

Bring a pragmatic engineering perspective focused on what we can actually build and ship."""
    },
    {
        "name": "Product Designer",
        "role": "Design",
        "perspective": "User experience, interaction design, usability, and design quality",
        "system_prompt": """You are a Product Designer responsible for crafting intuitive, delightful user experiences.

Your mission: Ensure the solution is usable, accessible, and provides a great user experience.

Focus on:
- User experience and interaction design
- Usability, learnability, and accessibility
- User flows and mental models
- Information architecture and navigation
- Visual design and brand consistency
- Edge cases and error states
- User research insights and validation

Analyze problems by asking:
- Is this intuitive for users?
- What is the user flow?
- Are there usability issues or friction points?
- Is it accessible to all users?
- How do we handle edge cases and errors?
- Does this match user mental models?

Bring a user-centered design perspective that ensures solutions are not just functional but delightful to use."""
    },
]

print(f"✅ Agent personas configured:")
print(f"   Design Critique Panel: {len(CRITIQUE_PANEL)} critics")
print(f"   Interdisciplinary Team: {len(EXPERT_TEAM)} experts")
print(f"\n💡 Edit these personas above to experiment with different perspectives!")

In [ ]:
# ========================================
# BENCHMARK CONFIGURATION
# ========================================

# Which benchmark to run
BENCHMARK_NAME = "gsm8k"  # 🔧 CHANGE: "gsm8k", "mmlu", "truthfulqa", "arc"

# How many tasks to evaluate (None = all, start with 10-20 for testing)
NUM_TASKS = 5  # 🔧 CHANGE: Start small, increase for full eval

# Random seed for reproducibility
SEED = 42

# Model configuration
PROVIDER = DEFAULT_PROVIDER
MODEL = DEFAULT_MODEL

# ========================================
# STRATEGIES TO COMPARE
# ========================================

# Define which strategies to test
# Each entry: (strategy_name, kwargs)
STRATEGIES_TO_TEST = [
    # Baseline
    ("single", {
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
    }),

    # Debate with 2 agents, 1 round
    ("debate", {
        "n_debaters": 2,
        "n_rounds": 1,
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
    }),

    # Debate with 3 agents, 2 rounds
    ("debate", {
        "n_debaters": 3,
        "n_rounds": 2,
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
    }),

    # Consensus (no judge)
    ("consensus", {
        "n_agents": 3,
        "n_rounds": 2,
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
    }),

    # Self-consistency
    ("self_consistency", {
        "n_samples": 5,
        "provider": PROVIDER,
        "model": MODEL,
        "temperature": 0.8
    }),

    # ========================================
    # CUSTOM MENTAL MODEL STRATEGIES (ACTIVE)
    # ========================================

    # Design Critique - 5 designer archetypes provide feedback
    # Designers: Systems, Visual Craft, AI Specialist, HCI, IDEO Design Thinking
    ("design_critique", {
        "n_iterations": 2,  # Number of critique/revision cycles
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
        # Uses default design panel (can customize with critique_panel param)
    }),

    # Interdisciplinary Team - Classic tech team collaboration
    # Team: Product Manager, Software Engineer, Product Designer
    ("interdisciplinary_team", {
        "refinement_rounds": 1,  # Number of refinement iterations
        "provider": PROVIDER,
        "model": MODEL,
        "verbose": False
        # Uses default tech team (can customize with expert_team param)
    }),
]

print(f"✅ Configuration:")
print(f"   Benchmark: {BENCHMARK_NAME}")
print(f"   Tasks: {NUM_TASKS if NUM_TASKS else 'all'}")
print(f"   Strategies to test: {len(STRATEGIES_TO_TEST)}")
print(f"   Model: {MODEL} ({PROVIDER})")

## Custom Mental Model Strategies

The benchmark framework now supports **design critique** and **interdisciplinary team** strategies from notebooks 05 and 06!

### Design Critique Strategy

**5 Designer Archetypes** provide structured feedback → Iterative refinement

**Default Design Panel:**
1. **Systems Designer** - Holistic design, interconnections, systemic coherence
2. **Visual Craft Specialist** - Presentation clarity, visual hierarchy, aesthetics
3. **AI Specialist** - AI/ML accuracy, capabilities, limitations, best practices
4. **Human-Computer Interaction Expert** - Usability, accessibility, cognitive ergonomics
5. **IDEO Design Thinking Facilitator** - Empathy, creative exploration, iteration

**Use cases:**
- Writing tasks (blog posts, documentation, creative writing)
- Design and product solutions
- Tasks where quality improvement through iteration is valuable
- When you want structured feedback on different quality dimensions

**Configuration:**
```python
("design_critique", {
    "n_iterations": 2,  # How many critique/revision cycles
    "provider": PROVIDER,
    "model": MODEL,
    # Optional: Custom critique panel (see strategies.py for examples)
})
```

---

### Interdisciplinary Team Strategy

**Classic Tech Team** (PM, Engineer, Designer) collaborates on solutions

**Default Tech Team:**
1. **Product Manager** - User needs, business value, priorities, success metrics
   - Mission: Create real user value while achieving business objectives
2. **Software Engineer** - Technical feasibility, architecture, scalability, implementation
   - Mission: Deliver technically sound, maintainable, scalable solutions
3. **Product Designer** - User experience, interaction design, usability, accessibility
   - Mission: Craft intuitive, delightful user experiences

**Use cases:**
- Complex problems requiring multiple perspectives
- Product and feature planning
- Strategic decision-making
- Tasks where PM/Eng/Design perspectives are valuable

**Configuration:**
```python
("interdisciplinary_team", {
    "refinement_rounds": 1,  # How many refinement cycles
    "provider": PROVIDER,
    "model": MODEL,
    # Optional: Custom expert team (see strategies.py for examples)
})
```

---

**Status:** Both strategies are **active by default** in the configuration above. To disable, comment them out in `STRATEGIES_TO_TEST`.

## Load Benchmark

In [ ]:
# Load benchmark
benchmark = load_benchmark(BENCHMARK_NAME)
tasks = benchmark.get_tasks(n=NUM_TASKS, seed=SEED)

print(f"📊 Loaded {len(tasks)} tasks from {BENCHMARK_NAME}")
print(f"\nSample task:")
print(f"  Input: {tasks[0].input[:100]}...")
print(f"  Expected: {tasks[0].expected}")
print(f"  Category: {tasks[0].category}")

# Show baseline scores from literature
baselines = get_baseline_scores(BENCHMARK_NAME)
if baselines:
    print(f"\n📈 Published baseline scores on {BENCHMARK_NAME}:")
    for model, score in sorted(baselines.items(), key=lambda x: x[1], reverse=True):
        print(f"   {model:20s}: {score:.1%}")

## Run Evaluation

Test each strategy on the benchmark tasks

In [ ]:
# Run evaluation for each strategy
results = []

for strategy_name, strategy_kwargs in STRATEGIES_TO_TEST:
    print(f"\n{'='*80}")
    print(f"🔄 Running: {strategy_name}")
    print(f"   Config: {strategy_kwargs}")
    print(f"{'='*80}\n")
    
    # Create experiment tracker
    config = ExperimentConfig(
        experiment_name=f"benchmark_{BENCHMARK_NAME}_{strategy_name}",
        task_type=BENCHMARK_NAME,
        strategy=strategy_name,
        provider=PROVIDER,
        model=MODEL,
        notes=f"Benchmark eval on {BENCHMARK_NAME}"
    )
    
    tracker = get_tracker()
    tracker.start_experiment(config)
    
    # Run on each task
    for task in tqdm(tasks, desc=f"{strategy_name}"):
        # Run strategy
        result = run_strategy(strategy_name, task.input, **strategy_kwargs)
        
        # Evaluate with benchmark-specific eval
        eval_scores = benchmark.evaluate(task, result.output)
        
        # Log result
        exp_result = ExperimentResult(
            config=config,
            task_input=task.input,
            output=result.output,
            latency_s=result.latency_s,
            tokens_in=result.tokens_in,
            tokens_out=result.tokens_out,
            cost_usd=result.cost_usd,
            eval_scores=eval_scores,
            eval_metadata={
                'task_id': task.id,
                'expected': task.expected,
                'benchmark': BENCHMARK_NAME
            }
        )
        
        tracker.log_result(exp_result)
        results.append({
            'strategy': strategy_name,
            'task_id': task.id,
            'accuracy': eval_scores.get('accuracy', 0),
            'latency_s': result.latency_s,
            'cost_usd': result.cost_usd or 0
        })
    
    # Finish experiment
    summary = tracker.finish_experiment()
    
    print(f"\n✅ {strategy_name} complete:")
    print(f"   Accuracy: {summary['eval_scores'].get('accuracy', {}).get('mean', 0):.1%}")
    print(f"   Avg latency: {summary['avg_latency_s']:.2f}s")
    print(f"   Total cost: ${summary['total_cost_usd']:.4f}")

print(f"\n{'='*80}")
print("✅ All strategies evaluated!")
print(f"{'='*80}")

## Results Analysis

In [ ]:
# Convert to DataFrame for analysis
df = pd.DataFrame(results)

# Aggregate by strategy
strategy_summary = df.groupby('strategy').agg({
    'accuracy': ['mean', 'std'],
    'latency_s': 'mean',
    'cost_usd': 'sum'
}).round(4)

print("\n📊 Strategy Comparison:")
print(strategy_summary)

# Sort by accuracy
accuracy_ranking = df.groupby('strategy')['accuracy'].mean().sort_values(ascending=False)
print("\n🏆 Accuracy Ranking:")
for i, (strategy, acc) in enumerate(accuracy_ranking.items(), 1):
    print(f"   {i}. {strategy:20s}: {acc:.1%}")

## Visualizations

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Accuracy by strategy
ax = axes[0, 0]
strategy_acc = df.groupby('strategy')['accuracy'].mean().sort_values(ascending=True)
strategy_acc.plot(kind='barh', ax=ax, color='steelblue')
ax.set_title(f'Accuracy on {BENCHMARK_NAME}', fontsize=14, weight='bold')
ax.set_xlabel('Accuracy')
ax.set_xlim(0, 1.0)
ax.axvline(x=0.5, color='red', linestyle='--', alpha=0.3, label='Random (if applicable)')
ax.grid(True, alpha=0.3)

# 2. Latency by strategy
ax = axes[0, 1]
strategy_latency = df.groupby('strategy')['latency_s'].mean().sort_values()
strategy_latency.plot(kind='barh', ax=ax, color='coral')
ax.set_title('Average Latency', fontsize=14, weight='bold')
ax.set_xlabel('Seconds')
ax.grid(True, alpha=0.3)

# 3. Cost by strategy
ax = axes[1, 0]
strategy_cost = df.groupby('strategy')['cost_usd'].sum().sort_values()
strategy_cost.plot(kind='barh', ax=ax, color='green')
ax.set_title('Total Cost', fontsize=14, weight='bold')
ax.set_xlabel('USD')
ax.grid(True, alpha=0.3)

# 4. Accuracy vs Latency scatter
ax = axes[1, 1]
strategy_stats = df.groupby('strategy').agg({
    'accuracy': 'mean',
    'latency_s': 'mean'
})
ax.scatter(strategy_stats['latency_s'], strategy_stats['accuracy'], s=200, alpha=0.6)
for strategy, row in strategy_stats.iterrows():
    ax.annotate(strategy, (row['latency_s'], row['accuracy']), 
                xytext=(5, 5), textcoords='offset points', fontsize=9)
ax.set_xlabel('Avg Latency (s)', fontsize=11)
ax.set_ylabel('Accuracy', fontsize=11)
ax.set_title('Accuracy vs Latency Tradeoff', fontsize=14, weight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'../experiments/benchmark_{BENCHMARK_NAME}_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Saved plot to: experiments/benchmark_{BENCHMARK_NAME}_comparison.png")

## Compare to Published Baselines

In [ ]:
# Compare your results to published baselines
baselines = get_baseline_scores(BENCHMARK_NAME)

if baselines:
    # Get your best strategy
    your_best = df.groupby('strategy')['accuracy'].mean().max()
    your_best_strategy = df.groupby('strategy')['accuracy'].mean().idxmax()
    
    print(f"\n📊 Comparison to Published Baselines on {BENCHMARK_NAME}:")
    print(f"\nYour best: {your_best_strategy} = {your_best:.1%}\n")
    
    # Create comparison DataFrame
    comparison_data = []
    
    # Add baselines
    for model, score in baselines.items():
        comparison_data.append({
            'Model/Strategy': model,
            'Accuracy': score,
            'Type': 'Published Baseline'
        })
    
    # Add your results
    for strategy, score in df.groupby('strategy')['accuracy'].mean().items():
        comparison_data.append({
            'Model/Strategy': f"{MODEL} ({strategy})",
            'Accuracy': score,
            'Type': 'Your Results'
        })
    
    comparison_df = pd.DataFrame(comparison_data).sort_values('Accuracy', ascending=False)
    
    # Plot comparison
    plt.figure(figsize=(12, 6))
    colors = ['steelblue' if t == 'Published Baseline' else 'coral' 
              for t in comparison_df['Type']]
    
    plt.barh(range(len(comparison_df)), comparison_df['Accuracy'], color=colors)
    plt.yticks(range(len(comparison_df)), comparison_df['Model/Strategy'])
    plt.xlabel('Accuracy', fontsize=12)
    plt.title(f'{BENCHMARK_NAME} - Your Results vs Published Baselines', 
              fontsize=14, weight='bold')
    plt.xlim(0, 1.0)
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='steelblue', label='Published Baseline'),
        Patch(facecolor='coral', label='Your Results')
    ]
    plt.legend(handles=legend_elements, loc='lower right')
    
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Insights:")
    print(f"   - Your best strategy achieved {your_best:.1%} accuracy")
    
    # Find closest baseline
    baseline_scores = list(baselines.values())
    closest_baseline = min(baseline_scores, key=lambda x: abs(x - your_best))
    closest_model = [k for k, v in baselines.items() if v == closest_baseline][0]
    
    print(f"   - Closest published baseline: {closest_model} ({closest_baseline:.1%})")
    
    if your_best > closest_baseline:
        print(f"   - ✅ You're {(your_best - closest_baseline):.1%} better!")
    else:
        print(f"   - Room for improvement: {(closest_baseline - your_best):.1%} gap")
else:
    print("No published baselines available for this benchmark.")

## Key Findings

### Quantitative Results

[Fill in after running]

- Best strategy: _____ (___% accuracy)
- Single-model baseline: ___% accuracy
- Improvement from multi-agent: ___%
- Latency overhead: ___x
- Cost overhead: $___

### Insights

- When did multi-agent help vs hurt?
- Was the cost/latency tradeoff worth it?
- How do we compare to published baselines?
- Which strategy is best for this benchmark?

### Next Steps

1. Test on different benchmarks
2. Tune debate/consensus parameters
3. Try with larger models
4. Analyze failure cases

## Error Analysis

Examine specific failures to understand where strategies struggle

In [ ]:
# Find tasks where all strategies failed
task_accuracy = df.groupby('task_id')['accuracy'].mean()
hard_tasks = task_accuracy[task_accuracy == 0].index.tolist()

if hard_tasks:
    print(f"\n❌ Tasks where all strategies failed: {len(hard_tasks)}")
    print(f"\nSample hard task:")
    hard_task = [t for t in tasks if t.id == hard_tasks[0]][0]
    print(f"  {hard_task.input[:200]}...")
    print(f"  Expected: {hard_task.expected}")
else:
    print("\n✅ At least one strategy got each task correct!")

# Find tasks where multi-agent helped most
single_results = df[df['strategy'] == 'single'].set_index('task_id')['accuracy']
multi_results = df[df['strategy'] != 'single'].groupby('task_id')['accuracy'].max()

improvement = multi_results - single_results
best_improvements = improvement.nlargest(3)

if len(best_improvements) > 0:
    print(f"\n✅ Tasks where multi-agent helped most:")
    for task_id, improvement in best_improvements.items():
        if improvement > 0:
            task = [t for t in tasks if t.id == task_id][0]
            print(f"\n  Task: {task.input[:100]}...")
            print(f"  Single: {single_results[task_id]:.0%} → Multi: {multi_results[task_id]:.0%}")
            print(f"  Improvement: +{improvement:.0%}")